# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.tail()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
558,rabo de peixe,37.8000,-25.5833,21.73,99,20,40,PT,1723170167
559,kuloy,61.0305,42.4925,13.36,98,99,35,RU,1723170168
560,majene,-3.5403,118.9707,27.61,69,56,113,ID,1723170169
561,anna regina,7.2644,-58.5077,26.99,79,60,82,GY,1723170170
562,cestos city,5.4600,-9.5828,24.04,80,100,194,LR,1723170171


In [21]:
city_data_df.describe

<bound method NDFrame.describe of                   City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0             tome acu  -2.4189  -48.1522     25.85        69          97   
1              dadukou  26.5479  101.7054     27.60        65          71   
2    port-aux-francais -49.3500   70.2167      2.39        71          60   
3                sinop  41.7711   34.8709     16.82        85           0   
4           whitehorse  60.7161 -135.0538     29.06        26           0   
..                 ...      ...       ...       ...       ...         ...   
558      rabo de peixe  37.8000  -25.5833     21.73        99          20   
559              kuloy  61.0305   42.4925     13.36        98          99   
560             majene  -3.5403  118.9707     27.61        69          56   
561        anna regina   7.2644  -58.5077     26.99        79          60   
562        cestos city   5.4600   -9.5828     24.04        80         100   

     Wind Speed Country        Date  
0  

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 563 entries, 0 to 562
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        563 non-null    object 
 1   Lat         563 non-null    float64
 2   Lng         563 non-null    float64
 3   Max Temp    563 non-null    float64
 4   Humidity    563 non-null    int64  
 5   Cloudiness  563 non-null    int64  
 6   Wind Speed  563 non-null    int64  
 7   Country     560 non-null    object 
 8   Date        563 non-null    int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 39.7+ KB


In [23]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1, 
    color="City", 
    alpha=0.5, 
    tiles="OSM", 
    width=700, 
    height=500, 
    title="City Humidity Map"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [24]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
243,shalqar,47.8333,59.6000,21.36,68,0,3,KZ,1723169802
451,gela,37.0803,14.2307,24.89,61,0,0,IT,1723170046


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
243,shalqar,KZ,47.8333,59.6000,68,
451,gela,IT,37.0803,14.2307,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row["Lat"], row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shalqar - nearest hotel: Бекнур
gela - nearest hotel: Archeo Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
243,shalqar,KZ,47.8333,59.6000,68,Бекнур
451,gela,IT,37.0803,14.2307,61,Archeo Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    width=700,
    height=500,
    title="Hotel Locations",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)